In [1]:
# Pick response for supervised prob
response = "state"

# df[response] = df[response].asfactor()

In [2]:
import h2o
import numpy as np
import math
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-linux64) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-linux64) (build 25.121-b15, mixed mode)
  Starting server from /home/rocket/anaconda2/h2o_jar/h2o.jar
  Ice root: /tmp/tmp2vNBIn
  JVM stdout: /tmp/tmp2vNBIn/h2o_rocket_started_from_python.out
  JVM stderr: /tmp/tmp2vNBIn/h2o_rocket_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster version:,3.10.0.9
H2O cluster version age:,"1 year, 2 months and 23 days !!!"
H2O cluster name:,H2O_from_python_rocket_ym7w1q
H2O cluster total nodes:,1
H2O cluster free memory:,3.464 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [3]:
df = h2o.import_file(path="../datasets/kickstarter-2018-reduced.csv")
print df.dim
print df.head
print df.tail
df.describe()

Parse progress: |█████████████████████████████████████████████████████████| 100%
[29999, 10]


category,main_category,currency,goal,launched_utc,state,backers,country,usd pledged,time_open
Poetry,Publishing,GBP,1000,1.4393e+09,failed,0,GB,0,1415
Narrative Film,Film & Video,USD,30000,1.50433e+09,failed,15,US,100,1438
Narrative Film,Film & Video,USD,45000,1.35795e+09,failed,3,US,220,1080
Music,Music,USD,5000,1.33195e+09,failed,1,US,1,721
Film & Video,Film & Video,USD,19500,1.436e+09,canceled,14,US,1283,1336
Restaurants,Food,USD,50000,1.45649e+09,successful,224,US,52375,840
Food,Food,USD,1000,1.41746e+09,successful,16,US,1205,480
Drinks,Food,USD,25000,1.45436e+09,failed,40,US,453,1078
Product Design,Design,USD,125000,1.39836e+09,canceled,58,US,8233,840
Documentary,Film & Video,USD,65000,1.40512e+09,canceled,43,US,6240.57,720


<bound method H2OFrame.head of >


category,main_category,currency,goal,launched_utc,state,backers,country,usd pledged,time_open
Poetry,Publishing,GBP,1000,1.4393e+09,failed,0,GB,0,1415
Narrative Film,Film & Video,USD,30000,1.50433e+09,failed,15,US,100,1438
Narrative Film,Film & Video,USD,45000,1.35795e+09,failed,3,US,220,1080
Music,Music,USD,5000,1.33195e+09,failed,1,US,1,721
Film & Video,Film & Video,USD,19500,1.436e+09,canceled,14,US,1283,1336
Restaurants,Food,USD,50000,1.45649e+09,successful,224,US,52375,840
Food,Food,USD,1000,1.41746e+09,successful,16,US,1205,480
Drinks,Food,USD,25000,1.45436e+09,failed,40,US,453,1078
Product Design,Design,USD,125000,1.39836e+09,canceled,58,US,8233,840
Documentary,Film & Video,USD,65000,1.40512e+09,canceled,43,US,6240.57,720


<bound method H2OFrame.tail of >
Rows:29999
Cols:10




,category,main_category,currency,goal,launched_utc,state,backers,country,usd pledged,time_open
type,enum,enum,enum,real,int,enum,int,enum,real,int
mins,0.0,0.0,0.0,1.0,3600.0,0.0,0.0,0.0,0.0,24.0
mean,NaN,NaN,NaN,39679.8923187,1411614526.6,NaN,104.256908564,NaN,6904.48511524,831.233841128
maxs,158.0,14.0,13.0,100000000.0,1514880718.0,5.0,85581.0,22.0,8596474.58,356809.0
sigma,NaN,NaN,NaN,743464.274157,61066107.2589,NaN,899.326567671,NaN,75466.6634392,2078.18963537
zeros,58,2266,622,0,0,3032,4275,51,5216,0
missing,0,0,0,0,0,0,0,0,304,0
0,Poetry,Publishing,GBP,1000.0,1439295148.0,failed,0.0,GB,0.0,1415.0
1,Narrative Film,Film & Video,USD,30000.0,1504327437.0,failed,15.0,US,100.0,1438.0
2,Narrative Film,Film & Video,USD,45000.0,1357950050.0,failed,3.0,US,220.0,1080.0


In [4]:
# Define predictors
predictors = df.columns
del predictors[5]
del predictors[5]
del predictors[6]
del predictors[2]
print predictors

[u'category', u'main_category', u'goal', u'launched_utc', u'country', u'time_open']


In [5]:
# Split for training
train, valid, test = df.split_frame(
    ratios = [0.6, 0.2],
    seed=4567,
    destination_frames=['train.hex','valid.hex','test.hex']
)

In [6]:
# Defualt gbm tree
gbm = H2OGradientBoostingEstimator()
gbm.train(x=predictors, y=response, training_frame=train)

# Show model summary
print gbm

gbm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1517078704861_1
Model Summary: 


,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,300.0,175886.0,3.0,5.0,4.9933333,4.0,32.0,24.483334




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.257845270127
RMSE: 0.507784669054
LogLoss: 0.73875060768
Mean Per-Class Error: 0.393843262451
Confusion Matrix: vertical: actual; across: predicted



canceled,failed,live,successful,suspended,undefined,Error,Rate
129.0,1289.0,2.0,381.0,0.0,0.0,0.9283731,"1,672 / 1,801"
6.0,7717.0,0.0,1626.0,0.0,0.0,0.1745641,"1,632 / 9,349"
0.0,0.0,137.0,0.0,0.0,0.0,0.0,0 / 137
3.0,2482.0,1.0,4054.0,0.0,0.0,0.3801223,"2,486 / 6,540"
0.0,52.0,0.0,14.0,9.0,0.0,0.88,66 / 75
0.0,0.0,0.0,0.0,0.0,173.0,0.0,0 / 173
138.0,11540.0,140.0,6075.0,9.0,173.0,0.3239834,"5,856 / 18,075"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.6760166
2,0.9272475
3,0.9973997
4,0.9999999
5,0.9999999
6,0.9999999


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2018-01-27 13:45:07,0.014 sec,0.0,0.8333333,1.7917595,0.5899862
,2018-01-27 13:45:08,0.554 sec,1.0,0.7999680,1.6142174,0.3970678
,2018-01-27 13:45:08,0.790 sec,2.0,0.7707097,1.4857274,0.3947994
,2018-01-27 13:45:08,1.009 sec,3.0,0.7449677,1.3872407,0.3934716
,2018-01-27 13:45:09,1.209 sec,4.0,0.7218990,1.3074429,0.3911480
---,---,---,---,---,---,---
,2018-01-27 13:45:11,3.826 sec,40.0,0.5164823,0.7614093,0.3321162
,2018-01-27 13:45:11,3.878 sec,41.0,0.5155131,0.7589595,0.3316736
,2018-01-27 13:45:11,3.930 sec,42.0,0.5145627,0.7563816,0.3298479
,2018-01-27 13:45:11,3.998 sec,43.0,0.5136922,0.7542007,0.3288520



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
category,7521.4404297,1.0,0.5765408
country,1760.7579346,0.2340985,0.1349673
goal,1345.2509766,0.1788555,0.1031175
launched_utc,1327.8417969,0.1765409,0.1017830
time_open,1077.9896240,0.1433222,0.0826311
main_category,12.5276918,0.0016656,0.0009603


In [7]:
perf = gbm.model_performance(valid)
perf


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.311036335163
RMSE: 0.55770631623
LogLoss: 0.90238431137
Mean Per-Class Error: 0.482144646613
Confusion Matrix: vertical: actual; across: predicted



canceled,failed,live,successful,suspended,undefined,Error,Rate
4.0,501.0,6.0,140.0,1.0,0.0,0.9938650,648 / 652
15.0,2404.0,7.0,692.0,1.0,1.0,0.2294872,"716 / 3,120"
0.0,7.0,42.0,3.0,0.0,0.0,0.1923077,10 / 52
5.0,1000.0,0.0,1101.0,0.0,0.0,0.4772080,"1,005 / 2,106"
1.0,23.0,0.0,6.0,0.0,0.0,1.0,30 / 30
0.0,0.0,0.0,0.0,0.0,51.0,0.0,0 / 51
25.0,3935.0,55.0,1942.0,2.0,52.0,0.4007653,"2,409 / 6,011"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.5992348
2,0.8948594
3,0.9928465
4,0.9996673
5,1.0000001
6,1.0000001


In [8]:
cv_gbm = H2OGradientBoostingEstimator(nfolds = 4, seed = 0xDEFCA)
cv_gbm.train(x=predictors, y=response, training_frame=train.rbind(valid))

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [9]:
# Show detailed summary of cross validation metrics
# Gives better idea of variance between the folds
cv_summary = cv_gbm.cross_validation_metrics_summary().as_data_frame()
print(cv_summary) # Full Summary
print(cv_summary.iloc[4])

                                  mean            sd  cv_1_valid  cv_2_valid  \
0                 accuracy    0.604703  0.0011979433   0.6050881  0.60653573   
1                      err  0.39529702  0.0011979433   0.3949119  0.39346427   
2                err_count     2380.25     4.1944904      2375.0      2384.0   
3                  logloss    0.886422  0.0031232962   0.8847267   0.8823848   
4      max_per_class_error         1.0           0.0         1.0         1.0   
5  mean_per_class_accuracy  0.51830935  0.0039948346   0.5108297   0.5247203   
6     mean_per_class_error  0.48169065  0.0039948346  0.48917034  0.47527972   
7                      mse  0.30709633  0.0010126876  0.30632415  0.30586603   
8                       r2  0.75983506   0.001312837   0.7622826   0.7579219   
9                     rmse  0.55416125   9.126302E-4   0.5534656   0.5530516   

   cv_3_valid  cv_4_valid  
0  0.60193366   0.6052544  
1  0.39806634   0.3947456  
2      2388.0      2374.0  
3  0.89

In [10]:
# Get cross-validated AUC through scoring combined holdout predictions.
perf_cv = cv_gbm.model_performance(xval=True)
perf_cv


ModelMetricsMultinomial: gbm
** Reported on cross-validation data. **

MSE: 0.3070925271
RMSE: 0.554159297585
LogLoss: 0.886409775502
Mean Per-Class Error: 0.481924424559
Confusion Matrix: vertical: actual; across: predicted



canceled,failed,live,successful,suspended,undefined,Error,Rate
17.0,1874.0,13.0,549.0,0.0,0.0,0.9930697,"2,436 / 2,453"
57.0,9640.0,18.0,2752.0,1.0,1.0,0.2268827,"2,829 / 12,469"
2.0,20.0,152.0,14.0,1.0,0.0,0.1957672,37 / 189
23.0,4082.0,8.0,4532.0,0.0,1.0,0.4758270,"4,114 / 8,646"
0.0,83.0,1.0,21.0,0.0,0.0,1.0,105 / 105
0.0,0.0,0.0,0.0,0.0,224.0,0.0,0 / 224
99.0,15699.0,192.0,7868.0,2.0,226.0,0.3952919,"9,521 / 24,086"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.6047081
2,0.8996513
3,0.9942291
4,0.999834
5,1.0000001
6,1.0000001


In [11]:
gbm_lucky = H2OGradientBoostingEstimator(
    # more trees the better if learning rate small
    # use 'more than enough' since we have early stop
    ntrees = 10000,
    
    # smaller learning rate is better
    learn_rate = 0.01,
    
#     Early stopping once validation AUC doesn't improve by at least 0.01% for 5 consecutive scoring events
    stopping_rounds = 5, stopping_tolerance = 1e-4, stopping_metric = "auto",
    
#     Sample 80% of columns per split
    sample_rate = 0.8,
    
#     fix random number generator seed
    seed = 1234,
#     score every 10 trees
    score_tree_interval = 10
)

gbm_lucky.train(x=predictors, y=response, training_frame=train, validation_frame=valid)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [12]:
perf_lucky = gbm_lucky.model_performance(valid)
perf_lucky


ModelMetricsMultinomial: gbm
** Reported on test data. **

MSE: 0.309634531814
RMSE: 0.556448139375
LogLoss: 0.898495497066
Mean Per-Class Error: 0.479039243218
Confusion Matrix: vertical: actual; across: predicted



canceled,failed,live,successful,suspended,undefined,Error,Rate
2.0,506.0,6.0,137.0,1.0,0.0,0.9969325,650 / 652
13.0,2428.0,6.0,671.0,1.0,1.0,0.2217949,"692 / 3,120"
0.0,6.0,43.0,3.0,0.0,0.0,0.1730769,9 / 52
5.0,1011.0,0.0,1090.0,0.0,0.0,0.4824311,"1,016 / 2,106"
0.0,23.0,0.0,7.0,0.0,0.0,1.0,30 / 30
0.0,0.0,0.0,0.0,0.0,51.0,0.0,0 / 51
20.0,3974.0,55.0,1908.0,2.0,52.0,0.3987689,"2,397 / 6,011"


Top-6 Hit Ratios: 


k,hit_ratio
1,0.6012311
2,0.8965231
3,0.9931792
4,0.9996673
5,1.0
6,1.0


In [13]:
h2o.shutdown()

H2O session _sid_baec closed.
